In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder 
        .appName("Sledgehammer") 
        .getOrCreate()  
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/03 06:06:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
import time
start = time.time()
spark.sql("select 1").collect()
end = time.time()

print(f"Time taken to run the query: {end - start} seconds")

Time taken to run the query: 0.03596687316894531 seconds


In [28]:
import os

In [4]:
import duckdb
cn = duckdb.connect()

In [14]:
start = time.time()
cn.execute("select 1").fetchall()
end = time.time()   

print(f"Time taken to run the query: {end - start} seconds")

Time taken to run the query: 0.0009810924530029297 seconds


In [33]:
# Generate dummy data using DuckDB SQL
rows = 500_000

lower_bound = "2020-01-01"
upper_bound = "2025-01-01"

sql = f"""
    select  
          t.row_id
        , cast(uuid() as varchar(30)) as txn_key
        , date '{lower_bound}' 
            + (random() * (date_diff('day', date '{lower_bound}', date '{upper_bound}')))::int as rpt_dt
        ,round(random() * 100,2) as some_val
    from generate_series(1,{rows}) t(row_id)
"""

# Write to Parquet
f_path = os.path.expanduser("~/dummy_data/dummy_data.parquet")

cn.execute(f"COPY ({sql}) TO '{f_path}' (FORMAT 'parquet')")


In [34]:
start = time.time()
cn.sql(f"select rpt_dt, count(*) from read_parquet('{f_path}') group by all order by 2 desc limit 30").fetchall()
end = time.time()

print(f"Time taken to run the query in duckdb: {end - start} seconds")

start = time.time()
spark.read.parquet(f_path).groupBy("rpt_dt").count().orderBy("count", ascending=False).collect()
end = time.time()

print(f"Time taken to run the query in Spark: {end - start} seconds")

Time taken to run the query in duckdb: 0.004390239715576172 seconds
Time taken to run the query in Spark: 0.23548316955566406 seconds
